In [1]:
import pandas as pd
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
import optuna
from optuna import Trial
from optuna.samplers import TPESampler

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

# from lightgbm import LGBMClassifier, log_evaluation, early_stopping
from sklearn.model_selection import cross_val_score

In [2]:
path_train = "./data/train_data_1.csv"
path_val = "./data/val_data_0.csv"

data_train = pd.read_csv(path_train)
# data_val = pd.read_csv(path_val)
data_test = pd.read_csv(path_val)

print(data_train.shape)
# print(data_val.shape)
print(data_test.shape)
data_test.head(10)

(2595852, 58)
(648963, 58)


,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,...,insert_time_minute,personal_rehabilitation_yn_0,personal_rehabilitation_yn_1,personal_rehabilitation_yn_2,personal_rehabilitation_complete_yn_0,personal_rehabilitation_complete_yn_1,personal_rehabilitation_complete_yn_2,personal_rehabilitation_complete_yn_3,company_enter_month_year,company_enter_month_month
0,1867362.0,2022-05-07 12:09:28,4.0,268.0,20000000.0,10.9,1.0,650628.0,1968.0,1.0,...,9,1,0,0,0,0,1,0,1970.0,1.0
1,1888879.0,2022-05-02 20:22:17,59.0,150.0,14000000.0,17.6,0.0,75483.0,1974.0,0.0,...,22,1,0,0,0,0,1,0,1970.0,1.0
2,1950464.0,2022-03-14 11:09:33,50.0,142.0,14000000.0,15.9,1.0,649072.0,1984.0,1.0,...,9,0,0,1,0,0,0,1,1970.0,1.0
3,642480.0,2022-03-01 11:57:40,35.0,98.0,9000000.0,15.9,0.0,499722.0,1972.0,0.0,...,57,0,0,1,0,0,0,1,1970.0,1.0
4,1444962.0,2022-05-16 00:35:23,21.0,196.0,3000000.0,12.5,0.0,126504.0,1992.0,1.0,...,35,1,0,0,0,0,1,0,1970.0,1.0
5,234972.0,2022-03-08 12:39:02,17.0,19.0,32000000.0,14.2,0.0,571769.0,1995.0,1.0,...,39,0,0,1,0,0,0,1,1970.0,1.0
6,947895.0,2022-05-12 15:28:24,28.0,217.0,2000000.0,15.3,0.0,492833.0,1975.0,1.0,...,28,1,0,0,0,0,1,0,1970.0,1.0
7,143231.0,2022-05-31 09:18:18,2.0,7.0,30000000.0,11.8,1.0,269734.0,1979.0,1.0,...,18,1,0,0,0,0,1,0,1970.0,1.0
8,1111284.0,2022-03-16 14:35:35,59.0,251.0,9000000.0,6.7,1.0,5200.0,1979.0,1.0,...,35,0,0,1,0,0,0,1,1970.0,1.0
9,612789.0,2022-03-11 23:39:34,13.0,123.0,38000000.0,17.6,0.0,699038.0,1967.0,0.0,...,39,0,0,1,0,0,0,1,1970.0,1.0


In [3]:
X_train = data_train.drop(["is_applied"], axis=1)
y_train = data_train["is_applied"]
# X_val = data_val.drop(["is_applied"], axis=1)
# y_val = data_val["is_applied"]
X_test = data_test.drop(["is_applied"], axis=1)
y_test = data_test["is_applied"]

# X_val = X_val.drop(["loanapply_insert_time", "insert_time", "company_enter_month"])
X_train = X_train.drop(["loanapply_insert_time"], axis=1)
X_train = X_train.drop(["insert_time"], axis=1)
X_train = X_train.drop(["company_enter_month"], axis=1)

X_test = X_test.drop(["loanapply_insert_time"], axis=1)
X_test = X_test.drop(["insert_time"], axis=1)
X_test = X_test.drop(["company_enter_month"], axis=1)

# y_val = y_val.drop(["loanapply_insert_time", "insert_time", "company_enter_month"])


In [4]:
def RF_objective(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 1000)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
   
    model = RandomForestClassifier(max_depth = max_depth, max_leaf_nodes = max_leaf_nodes,n_estimators = n_estimators,n_jobs=3,random_state=25)

    
    model.fit(X_train, y_train)    
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1")
    f1_mean = score.mean()

    return f1_mean
    
#Execute optuna and set hyperparameters
RF_study = optuna.create_study(direction='maximize')
RF_study.optimize(RF_objective, n_trials=10)



[I 2022-10-12 13:13:05,025] A new study created in memory with name: no-name-4228cc5f-45ef-4994-b1df-c8926dfc7097
[W 2022-10-12 13:31:30,237] Trial 0 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "/Users/gimtaeyong/opt/anaconda3/lib/python3.9/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/nk/2nd0q1gd6d79970f37rc0sw80000gn/T/ipykernel_17182/5156979.py", line 10, in RF_objective
    score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1")
  File "/Users/gimtaeyong/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 509, in cross_val_score
    cv_results = cross_validate(
  File "/Users/gimtaeyong/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 267, in cross_validate
    results = parallel(
  File "/Users/gimtaeyong/opt/anaconda3/lib/python3.9/site-packages/joblib/parallel.py

KeyboardInterrupt: 

In [ ]:
#Create an instance with tuned hyperparameters
optimized_RF = RandomForestClassifier(max_depth = RF_study.best_params['max_depth'], max_leaf_nodes = RF_study.best_params['max_leaf_nodes'],
                                      n_estimators = RF_study.best_params['n_estimators'],n_jobs=2,random_state=25)
model.fit(train_X, train_y)

In [ ]:
train_preds = optimized_RF.predict(train_X)
f1 = f1_score(train_preds,train_y)
f1

In [ ]:
# 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
optimized_RF = RandomForestClassifier(n_estimators=474, max_depth=1, max_leaf_nodes=552, random_state=25)